In [221]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.regression import LabeledPoint
from sklearn import metrics
from numpy import array
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from pyspark.mllib.classification import SVMWithSGD, SVMModel
sc = SparkContext(appName="ClassificationOfWineDataset")

In [222]:
#getting data
data = sc.textFile("D://NEU - Big Data and Intelligent Analytics/Assignment 1/winequality-white-classification-parsed format.csv")

In [223]:
def line_to_field(line):
    #space_separated_line = line.replace(';', ' ')
    string_array = line.split(',')
    #float_array = map(float, string_array)
    #float_array[11] = np.where(float_array[10] > 5.0, 1, 0)
    #float_array[11]=(float_array[10] > 5) 
    return LabeledPoint(string_array[0], string_array[1].split(' '))


In [224]:
parsedData = data.map(line_to_field)

In [225]:
#Spliting data into training and test data
(training, test) = parsedData.randomSplit([0.7, 0.3])

In [358]:
# Build the model
model = SVMWithSGD.train(training, iterations=200,step=0.01,regParam= 2)

In [359]:
# Evaluating the model on training data
labelsAndPreds = training.map(lambda p: (p.label, model.predict(p.features))).cache()
accuracy_training = labelsAndPreds.filter(lambda (v, p): v == p).count() / float(training.count())
print("Accuracy(training) = " + str(accuracy_training))
x = np.array(labelsAndPreds.collect())

Accuracy(training) = 0.701461988304


In [360]:
# Evaluating the model on test data
labelsAndPreds1 = test.map(lambda p: (p.label, model.predict(p.features))).cache()
accuracy_test = labelsAndPreds1.filter(lambda (v, p): v == p).count() / float(test.count())
print("Accuracy(test) = " + str(accuracy_test))
y = np.array(labelsAndPreds1.collect())

Accuracy(test) = 0.69350473613


In [361]:
#clearing the threshold for getting scores as prediction.
model.clearThreshold()

In [362]:
#making an array of true lables and its predicted scores for training data
scoreandlabels = np.array(training.map(lambda p: (p.label, model.predict(p.features))).collect())
print "Training Area under curve  = " + str(roc_auc_score(scoreandlabels[:,0],scoreandlabels[:,1]))
print "Training Area under Precision-Recall = " + str(average_precision_score(scoreandlabels[:,0],scoreandlabels[:,1]))

Training Area under curve  = 0.680051178758
Training Area under Precision-Recall = 0.334218228414


In [363]:
#making an array of true lables and its predicted scores for test data
scoreandlabels1 = np.array(test.map(lambda p: (p.label, model.predict(p.features))).collect())
print "Test Area under curve  = " + str(roc_auc_score(scoreandlabels1[:,0],scoreandlabels1[:,1]))
print "Test Area under Precision-Recall = " + str(average_precision_score(scoreandlabels1[:,0],scoreandlabels1[:,1]))

Test Area under curve  = 0.645190473014
Test Area under Precision-Recall = 0.316384588629


In [364]:
#Making confusion matrix for training data
confusion_matrix(x[:,0],x[:,1])

array([[2054,  631],
       [ 390,  345]])

In [366]:
#Making confusion matrix for test data
confusion_matrix(y[:,0],y[:,1])

array([[887, 266],
       [187, 138]])

In [367]:
#checking the weights for the various features
model.weights

DenseVector([0.0119, 0.0004, 0.0004, -0.0252, -0.0002, 0.0357, -0.017, 0.0025, 0.0095, 0.002, 0.068])

In [220]:
sc.stop()